# Probabilistic Time Series Analysis

## Week 9: Introduction to Gaussian Processes

You will need the following new Python package. Install it either with `conda install -c conda-forge` if you use the Anaconda environment or `pip install`.

    GPy

Places where you are supposed to fill in code are marked

    #
    # TODO: some instructions
    # 
    
The rest of the code we will run and discuss if time permits, otherwise try it out at home and try to answer the questions mentioned in the text boxes for yourself.

### Please turn in the code before 11/21/2018 5:20pm. 

### Your work will be evaluated based on the code and plots. You don't need to write down your answers to other questions in the text blocks, just think them over.

### Title your submission file `lab9-student-[YOUR NET ID].ipynb`.

# Setup

In [ ]:
import numpy as np
import GPy

%matplotlib inline

# I. Basic Example

In [ ]:
# NOTE: Read this function carefully! You will modify it in the following sections.

def uniform_samples(x_min, x_max, n_samples, f, noise_scale):
    """Generates f(x) + noise for x uniformly distributed in [x_min, x_max]."""
    X = np.random.uniform(x_min, x_max, size=(n_samples, 1))
    Y = f(X) + np.random.normal(scale=noise_scale, size=(n_samples, 1))
    return X, Y

In [ ]:
# Build sample points and values:
X, Y = uniform_samples(-3.0, 3.0, 20, np.sin, 0.05)

kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(X, Y, kernel)
# This step estimates maximum likelihood values for the kernel parameters:
_ = m.optimize()

_ = m.plot(plot_density=True, plot_limits=[-5.0, 5.0])
_ = m.plot_samples(samples=10, plot_limits=[-5.0, 5.0])

In [ ]:
# NOTE: Now we'll draw data that's just on a line, but with more noise.
X, Y = uniform_samples(-3.0, 3.0, 20, lambda x: x, 0.8)

kernel = GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=1.0)
m = GPy.models.GPRegression(X, Y, kernel)
_ = m.optimize()

_ = m.plot(plot_density=True, plot_limits=[-5.0, 5.0])
_ = m.plot_samples(samples=10, plot_limits=[-5.0, 5.0])

# NOTE: Run this a few times and see if you notice signs of overfitting.

# II. Gaps in Data Coverage

In [ ]:
#
# TODO: Write a version of uniform_samples where there are no samples in the middle half
# of the interval [x_min, x_max], i.e. in the subinterval [x_min + (x_max - x_min) / 4, 
# x_max - (x_max - x_min) / 4]. What happens to the estimates in this region?
#

def gapped_samples(x_min, x_max, n_samples, f, noise_scale):
    pass

In [ ]:
# Make plots like in the previous section but replacing uniform_samples with gapped_samples. What changes?

# III. Choice of Kernel

In [ ]:
# Build sample points and values:
X, Y = uniform_samples(-10.0, 10.0, 100, np.sin, 0.5)

In [ ]:
#
# TODO: Using the single set of example data above, try repeating the previous steps (one set of plots per cell)
# with the following alternate kernels (read the documentation and use some sensible initial parameter settings):
#
# - GPy.kern.RBF (like before)   :   squared exponential
# - GPy.kern.RatQuad             :   rational quadratic
# - GPy.kern.Linear              :   linear
# - GPy.kern.Brownian            :   covariance of Brownian motion
# - GPy.kern.PeriodicExponential :   periodic
#
# To do this, edit the line `kernel = GPy.kern.SOMETHING(input_dim=1, ...)`
#
# For the last one, first try the parameter `period=6.0`, close to the actual period of our data, and then try
# systematically decreasing this parameter until the model starts failing.

# If you're interested, you can read more about these choices here:
# https://www.cs.toronto.edu/~duvenaud/cookbook/
#

# IV. A Challenging Example

Now we'll try to design a Gaussian process to model a difficult set of data. Don't worry about getting perfect results---it's hard!---but do your best.

In [ ]:
def markov_jump_samples(x_min, x_max, n_samples, rate, step_scale, noise_scale):
    jump_times = []
    last_time = float(x_min)
    while last_time <= x_max:
        last_time += np.random.exponential(rate)
        jump_times.append(last_time)
    jump_times = np.array(jump_times)
    process_values = np.arange(len(jump_times) + 1) * step_scale
    process_values = process_values - np.mean(process_values)
    
    X = np.random.uniform(x_min, x_max, size=(n_samples, 1))
    Y = np.zeros((n_samples, 1))
    for i in range(n_samples):
        x = X[i, 0]
        ix = np.searchsorted(jump_times, x)
        Y[i, 0] = process_values[ix] + np.random.normal(scale=noise_scale)
        
    return X, Y

# NOTE: Take a look at this function and try to understand what it's doing. When you think you know, make sure
# this is consistent with the plots below.

In [ ]:
# Seed to give a deterministic example:
np.random.seed(7)
X, Y = markov_jump_samples(-40.0, 40.0, 100, 10.0, 10.0, 0.05)

In [ ]:
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(X, Y, kernel)
_ = m.optimize()
_ = m.plot(plot_density=True)

In [ ]:
#
# TODO: Notice two bad phenomena in the above result:
#   1. The mean of the fit is not as "sharp" as the generated data at the stepping points.
#   2. In long flat regions like between -20 and 0, there is "ringing" where the mean fluctuates while the data is flat.
# Try to find a choice of kernel that improves on the RBF in this case. Present the results of at least two other
# kernel choices from GPy.
# 

# References

The beginning of this notebook is based on:

https://github.com/savinlab/gp_intro/blob/master/gp_regression_basic.ipynb